# Cohaga DS Task

### Data import and Analysis

In [1]:
import pandas as pd

In [32]:
data_df = pd.read_csv("task_1_google_maps_comments.csv")
print(data_df.info)
print(data_df.axes)

<bound method DataFrame.info of       google_maps_profile_id  review_id   
0                     624932    6386691  \
1                     624932    6386692   
2                     624932    6386693   
3                     624932    6386694   
4                     624932    6386695   
...                      ...        ...   
9995                  151741    3027281   
9996                  151741    3027284   
9997                  151741    3027288   
9998                  151741    3027291   
9999                  151741    3027293   

                                                 review  rating   
0     Sehr leckeres Essen... Personal ist zudem sehr...       5  \
1     Auf 800 m auf der Sonnenterrasse Heiden treffe...       5   
2     Sehr feines Libanesisches Essen. Ein Hauch vom...       5   
3                                           Gutes Essen       5   
4     Toller Geheimtipp.\nNicht viel Platz dafür ein...       5   
...                                              

In [33]:
sentences = data_pd.loc[:, ['google_maps_profile_id', 'review']]
sentences.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   google_maps_profile_id  10000 non-null  int64 
 1   review                  10000 non-null  object
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


Check languages

In [62]:
import fasttext
model = fasttext.load_model('lid.176.bin')
#print[review.replace('\n', ' ') for review in sentences.review])
lang_list = [model.predict(review.replace('\n', ' '), k=1)[0][0] for review in sentences.review]
sentences['language'] = lang_list


In [82]:
print(set(lang_list))
for l in set(lang_list):
    print(l, sentences[sentences.language == l].iloc[0])
sentences.language.value_counts()

{'__label__la', '__label__el', '__label__sl', '__label__en', '__label__cs', '__label__sk', '__label__bn', '__label__fi', '__label__sr', '__label__da', '__label__ast', '__label__it', '__label__ceb', '__label__hu', '__label__he', '__label__bg', '__label__ja', '__label__mn', '__label__nl', '__label__no', '__label__id', '__label__ca', '__label__als', '__label__th', '__label__sv', '__label__ko', '__label__ms', '__label__vi', '__label__uk', '__label__pl', '__label__br', '__label__ia', '__label__pt', '__label__es', '__label__nds', '__label__ro', '__label__ru', '__label__oc', '__label__nn', '__label__zh', '__label__eu', '__label__de', '__label__fa', '__label__sh', '__label__tr', '__label__hr', '__label__gl', '__label__ar', '__label__eo', '__label__fr'}
__label__la google_maps_profile_id          52807
review                      Tiptopp👌😍
language                  __label__la
Name: 1720, dtype: object
__label__el google_maps_profile_id             579787
review                    Είναι τέλειο.

language
__label__de     5808
__label__en     1728
__label__fr      991
__label__it      951
__label__es       99
__label__nl       62
__label__pt       52
__label__ar       51
__label__pl       29
__label__tr       24
__label__als      22
__label__ko       20
__label__sv       17
__label__ru       16
__label__zh       15
__label__ja       14
__label__hu        8
__label__da        7
__label__cs        6
__label__fi        6
__label__ro        6
__label__sl        5
__label__ca        5
__label__ceb       5
__label__no        5
__label__vi        4
__label__bg        4
__label__id        4
__label__uk        3
__label__eo        3
__label__he        3
__label__hr        3
__label__sr        2
__label__oc        2
__label__th        2
__label__ms        2
__label__el        2
__label__sk        2
__label__fa        1
__label__br        1
__label__ast       1
__label__nds       1
__label__gl        1
__label__mn        1
__label__nn        1
__label__eu        1
__label__la        1
__la